In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string

2023-12-07 23:14:33.242202: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 23:14:33.675243: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 23:14:33.675292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 23:14:33.725803: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 23:14:33.831569: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 23:14:33.833295: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
df=pd.read_csv('one_output_dataset.csv')
df

,speech,emotion
0,From the moment that the French defenses at Se...,anger
1,We observe today not a victory of party but a ...,joy
2,"Your Majesties, Your Highnesses, Distinguished...",trust
3,I am honored to be with you today at your comm...,anticipation
4,"Honorable UN Secretary General Mr Ban Ki-moon,...",sadness
5,It is with a profound sense of humility that I...,sadness
6,My message is that we'll be watching you.\r\n\...,fear
7,"Hello everybody. You know, Michelle and I have...",joy
8,"Your Majesties, Your Royal Highness, Excellenc...",anticipation
9,"Five score years ago, a great American, in who...",trust


In [4]:
dict={'anger':1, 'joy':2, 'trust':3, 'anticipation':4, 'sadness':5, 'fear':6}
df=df.replace(dict)
df

,speech,emotion
0,From the moment that the French defenses at Se...,1
1,We observe today not a victory of party but a ...,2
2,"Your Majesties, Your Highnesses, Distinguished...",3
3,I am honored to be with you today at your comm...,4
4,"Honorable UN Secretary General Mr Ban Ki-moon,...",5
5,It is with a profound sense of humility that I...,5
6,My message is that we'll be watching you.\r\n\...,6
7,"Hello everybody. You know, Michelle and I have...",2
8,"Your Majesties, Your Royal Highness, Excellenc...",4
9,"Five score years ago, a great American, in who...",3


In [5]:
#preprocess transcript

# download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# initialize a PorterStemmer
# stemmer = PorterStemmer()

def preprocess_text(text):
    # lowercase the text
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # tokenize the text
    words = word_tokenize(text)
    # remove stopwords and stem the words
    words = [word for word in words if word not in stopwords.words('english')]
    # join the words back into a string
    text = ' '.join(words)

    return text

df['speech'] = df['speech'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /home/arjun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/arjun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df['speech']

0    moment french defenses sedan meuse broken end ...
1    observe today victory party celebration freedo...
2    majesties highnesses distinguished guests comr...
3    honored today commencement one finest universi...
4    honorable un secretary general mr ban kimoon r...
5    profound sense humility accept honor chosen be...
6    message well watching wrong shouldnt back scho...
7    hello everybody know michelle really milking g...
8    majesties royal highness excellencies distingu...
9    five score years ago great american whose symb...
Name: speech, dtype: object

In [7]:
#pip install sentencepiece
tokenizer=AlbertTokenizer.from_pretrained('albert-large-v2')

model=TFAlbertForSequenceClassification.from_pretrained('albert-large-v2',num_labels=6)

2023-12-07 23:15:16.614625: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 23:15:16.615268: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All PyTorch model weights were used when initializing TFAlbertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifie

In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [9]:
#split in train and test in text and labels
train_texts = train['speech'].tolist()
train_labels = train['emotion'].tolist()

test_texts = test['speech'].tolist()
test_labels = test['emotion'].tolist()

In [10]:
#tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

In [11]:
#convert features and labels to tensors for both train and test
train_features = {key: tf.convert_to_tensor(val) for key, val in train_encodings.items()}
train_labels = tf.convert_to_tensor(train_labels)

test_features = {key: tf.convert_to_tensor(val) for key, val in test_encodings.items()}
test_labels = tf.convert_to_tensor(test_labels)

In [12]:
#prepare the training and testing dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
train_dataset = train_dataset.shuffle(10000).batch(1)

test_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels))
test_dataset = test_dataset.batch(1)

In [13]:
#define model metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

In [15]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [16]:
model.summary()

Model: "tf_albert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 albert (TFAlbertMainLayer)  multiple                  17683968  
                                                                 
 dropout_4 (Dropout)         multiple                  0         
                                                                 
 classifier (Dense)          multiple                  6150      
                                                                 
Total params: 17690118 (67.48 MB)
Trainable params: 17690118 (67.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
#fit model to train dataset
model.fit(train_dataset, epochs=3, validation_data=test_dataset, batch_size=1)

Epoch 1/3
5/8 [=================>............] - ETA: 17s - loss: 1.8760 - accuracy: 0.0000e+00

2023-12-07 23:17:26.450075: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 6 which is outside the valid range of [0, 6).  Label values: 6


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/home/arjun/miniconda3/envs/ml/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  F

In [15]:
model.save('first_model.keras')

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [14]:
loaded_model = tf.keras.models.load_model('first_model.keras')

/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


ValueError: Layer 'pooler' expected 0 variables, but received 2 variables during loading. Expected: []

In [32]:
# test_features
test_input={'input_ids': test_features['input_ids'][0], 'token_type_ids': test_features['token_type_ids'][0], 
            'attention_mask':  test_features['attention_mask'][0]}
test_input

{'input_ids': <tf.Tensor: shape=(512,), dtype=int32, numpy=
 array([    2,  3957,  3302,   134,  1455,  1389, 18249,    97,   116,
          270,  2144,   768,   340,   182,   461,  3560,  2594,   255,
         1376,  7024,  6746,  1868,  4292,    13, 14228,    49,   833,
          768,   797,    53,  5419,    49,    53,  3560,  2594,   255,
         6586,  3560,  2594,   255,  1327,    13,  2291,  1523, 12860,
         9470,  2552,    18,  2348,  1619,  8870,   930,   875,  1768,
           49,  5873,    13,    62,  8766,    13, 27083,  2232,  7024,
          712,   159,    13,    18, 18049,   150,  4282,  1207,  7024,
           13, 13391,   291,   361,   229,   340,   395,   154,   511,
         4827,   242,  7176,  1982,   376,   174,   130,  1694,  2064,
          395,   990,   182,  1369,  7773,  6897,  1161,  2950,    40,
         6702,   259,    44, 17837,   683,  1210,  4014, 25955,   721,
          174,  1025,  7476,   601,    83,  3215,   302, 13735,    44,
        17837,   

In [35]:
loaded_model.predict(test_input)

14/16 [=========================>....] - ETA: 0s

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.